In [1]:
!pip install -U transformers datasets sentencepiece -q

from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset
import torch

dataset = load_dataset("cnn_dailymail", "3.0.0")
train_data = dataset["train"].select(range(200))
val_data = dataset["validation"].select(range(50))

#model
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

#preprocess
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    labels = tokenizer(text_target=examples["highlights"], max_length=128, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_data.map(preprocess_function, batched=True)
tokenized_val = val_data.map(preprocess_function, batched=True)

#training args
training_args = TrainingArguments(
    output_dir="./t5-finetuned-simple",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    logging_steps=10,
    save_steps=1000,
    save_total_limit=1,
    learning_rate=2e-5,
    report_to="none",  #  disables W&B and TensorBoard
)
#value error fixing
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

#trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    data_collator=data_collator,
)

trainer.train()
trainer.save_model("./t5-finetuned-simple")
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

text = """
The Apollo 11 mission was the first human spaceflight to land on the Moon.
Neil Armstrong and Buzz Aldrin landed the lunar module Eagle on July 20, 1969,
while Michael Collins remained in lunar orbit. Armstrong became the first person to walk on the Moon.
"""

input_text = "summarize: " + text
input_ids = tokenizer.encode(input_text, return_tensors="pt").to(device)
summary_ids = model.generate(
    input_ids,
    max_length=80,
    min_length=20,
    length_penalty=2.0,
    num_beams=4,
)

summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print("\nGenerated Summary:\n", summary)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 11.0 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

3.0.0/train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

3.0.0/train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Step,Training Loss
10,2.837500
20,2.780400
30,2.544400
40,2.492800
50,2.433200
60,2.634900
70,2.635300
80,2.477900
90,2.465800
100,2.350200



Generated Summary:
 the Apollo 11 mission was the first human spaceflight to land on the Moon. Neil Armstrong and Buzz Aldrin landed the lunar module Eagle on July 20, 1969.
